### Dash is a dashboarding tool from the guys at Plotly
It uses React components to create interactive dashboards in the browser.\
This example is based on Simen Svenkeruds experiments in freg.\
CSS from the Mimir-project and some input from this youtube-video: https://www.youtube.com/watch?v=hSPmj7mK6ng

In [ ]:
import pandas as pd
import dapla as dp

In [ ]:
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
from jupyter_dash import JupyterDash
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

In [ ]:
# This might struggle to rerun, if you do not restart your server in the Server Hub Control
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
# Kanskje unødvendig, men greit å ha om man sliter med å ta ned serveren etter at den er startet
@classmethod
def _terminate_server_for_port(cls, host, port):
    shutdown_url = "http://{host}:{port}/_shutdown_{token}".format(
        host=host, port=port, token=JupyterDash._token
    )
    try:
        response = requests.get(shutdown_url)
    except Exception as e:
        pass

In [ ]:
# Les eksempeldatasett fra dapla
diam = dp.read_pandas("/felles/kurs/helt_python/diamonds")

In [ ]:
# Generer liste med "options" til dropdownen lengre nede, må være formatert i dicts
diam_colors = pd.unique(diam['color'])
drop_options = [{'label': 'Alle', 'value' : 'Alle'}]
for color in diam_colors:
    tmp_dict = {"label" : color, "value": color}
    drop_options.append(tmp_dict)
#drop_options

In [ ]:
# Opprett app-objektet
app = JupyterDash(__name__)

server = app.server

# Her er alle html-elementene som inngår i Dash-dashboardet.
# Finn passende "className" fra style.css som gjerne kan tas herfra 
# https://github.com/statisticsnorway/ssb-component-library/blob/master/lib/bundle.css
app.layout = html.Div([
    html.H1('Diamanter Dash-eksempel', className = "ssb-title"),
    
    
    dcc.Dropdown('slct_color',
                 # Denne bruker da settet med options vi lagde i cellen lengre opp
                 options = drop_options,
                 multi = False,
                 value = "All",
                 style = {'width' : '40%'},
                 className = "ssb-dropdown"
                ),
    
    html.Div(id = 'output_container', children =[]),
    html.Br(),
    
    dcc.Graph(id="diam_scatter", figure={})    
    
])

In [ ]:
# Dette er en decorator, som går "utenpå" funksjonen under, 
# som både lager grafen, og tar imot oppdateringer ift. input
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='diam_scatter', component_property='figure')],
    [Input(component_id='slct_color', component_property='value')]
)
def update_graph(option_slctd):
    #print(option_slct)
    #print(type(option_slctd))
    
    container = f"Du har valgt fargen: {option_slctd}"
    
    dff = diam.copy()
    
    # Filtrer om noe annet enn "All" er valgt 
    if option_slctd != "Alle":
        dff = dff[dff['color'] == option_slctd]
        
    
    fig = px.scatter(
        data_frame = dff,
        x = 'price',
        y = 'carat',
        color = 'color',
        # SSB-fargeskala
        color_discrete_sequence=["#1a9d49", "#3396d2", "#9582bb", "#f26539", "#d2bc2a", 
                                 "#274247", "#90cc93", "#0757450", "#143f90", "#472f91",
                                 "#93180a", "#9a7b1c", "#6f9090"]
    )
    
    return container, fig

In [ ]:
app.run_server(mode='external', port=8060)

In [ ]:
# This isnt really enough to rerun the scripts, as there will be issues with regestering outputs with the callback decorator etc.
#app._terminate_server_for_port("localhost", 8060)